### Import Libraries

In [271]:
# Import twitter dependencies
import tweepy
from config import *

In [272]:
# Import transformation dependencies
import pandas as pd
from datetime import datetime, date
import json
import re 

In [273]:
# For sentiment and subjectivity analysis
from textblob import TextBlob

In [274]:
# For stats 
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html
from scipy import stats

# Tweepy Setup

In [275]:
# Tweepy Setup
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [345]:
# Tweepy search parameters
search_terms = ['covid+vaccine OR covid-19+vaccine OR pfizer OR pfizer-biontech OR pfizerbiontech OR moderna OR astrazeneca OR astra+zeneca -filter:retweets']
end_date = date(2021,3,11)
max_id ='1370107817701609474'
limit = 1200 # this is my 15 minute limit :'( 

# Save files
output_csv ='csvs/tw72.csv'
output_json ='json/raw72.json'

# Functions

Collect and save data

In [ ]:
def GetTweets(maxi_id, search_terms, limit):
    data = []
    raw = []
    tweets = tweepy.Cursor(api.search, max_id = max_id, lang ='en', q=search_terms, tweet_mode='extended').items(limit)
    for t in tweets: 
        tweet ={
            'created' :t.created_at,
            'id_str' : t.id_str,
            'text' : t.full_text,
            'user_id' :t.user.id,
            'screen_name': t.user.screen_name,
            'location': t.user.location,
            'followers_count': t.user.followers_count,
            'user_favourites_count': t.user.favourites_count,
            'time_zone' :t.user.time_zone,
            'geo_enabled' :t.user.geo_enabled,
            'verified' :t.user.verified,
            'status_count': t.user.statuses_count,
            'geo':t.geo,
            'coords':t.coordinates,
            'retweet_count': t.retweet_count,
            'tweet_favourite_count' : t.favorite_count
        }
        
        # add sorted data to one list 
        data.append(tweet)
        # keep raw data in case I realize later that I've removed something important 
        raw.append(t)
        
    # Add parsed data to dataframe, convert to csv
    tweets_df = pd.DataFrame(data)
           
return tweets_df, raw

In [ ]:
def CreateJSON():
    # Save raw data to json 
    with open(output_json, 'w') as file:
    for resp in raw:
        jso = json.dumps(resp._json,sort_keys=True, indent=3)
        file.write(jso)
    

In [ ]:
# Makes tweets more readable 
def CleanTweets(txt): 
    # remove @ username 
    # txt = re.sub(r'@[A-Za-z0-9)]+','', txt)
    # this removes names, which are sometimes vaccine names, so not good for my filter

    # remove @ from username
    txt = txt.replace('@','')    
    
    # remove RT    
    txt = re.sub(r'RT[\s]+','',txt)
    
    # remove # but leave txt
    txt = re.sub(r'#','', txt)
    
    # remove hyperlinks
    txt = re.sub(r'https?:\/\/\S+', '', txt)
    return txt

In [ ]:
# to get subjectivity 
def RateSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

In [ ]:
# to get polarity
def RatePolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [ ]:
# add worded sentiment based on polarity score
def GetSentiment(num):
    if num < 0: 
        return 'negative' 
    elif num == 0:
        return 'neutral'
    else:
        return 'positive'

In [1]:
# Create Columns
def RateTweets(df):
    df = df.drop_duplicates()
    df['text'] = df['text'].apply(CleanTweets)
    df['Subjectivity'] = df['text'].apply(RateSubjectivity)
    df['Polarity'] = df['text'].apply(RatePolarity)
    df['Sentiment'] = df['Polarity'].apply(GetSentiment)
    return df 

In [346]:
# Pulls data from api and returns 2 lists:
    # one which is parsed for selected variables
    # one with all the raw data

data = []
raw = []
tweets = tweepy.Cursor(api.search, max_id = max_id, lang ='en', q=search_terms, tweet_mode='extended').items(limit)
for t in tweets: 
    tweet ={
        'created' :t.created_at,
        'id_str' : t.id_str,
        'text' : t.full_text,
        'user_id' :t.user.id,
        'screen_name': t.user.screen_name,
        'location': t.user.location,
        'followers_count': t.user.followers_count,
        'user_favourites_count': t.user.favourites_count,
        'time_zone' :t.user.time_zone,
        'geo_enabled' :t.user.geo_enabled,
        'verified' :t.user.verified,
        'status_count': t.user.statuses_count,
        'geo':t.geo,
        'coords':t.coordinates,
        'retweet_count': t.retweet_count,
        'tweet_favourite_count' : t.favorite_count
    }
    # add sorted data to one list 
    data.append(tweet)
    # keep raw data in case I realize later that I've removed something important 
    raw.append(t)
print('done!')

done!


In [347]:
# Add parsed data to dataframe, convert to csv
tweets_df = pd.DataFrame(data)

# Save raw data to json 
with open(output_json, 'w') as file:
    for resp in raw:
        jso = json.dumps(resp._json,sort_keys=True, indent=3)
        file.write(jso)
tweets_df.tail(2)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count
1198,2021-03-11 20:17:15,1370106565190094849,RT @cantpretendtoo: Wife’s Uncle got the Covid...,1250179015215898624,Rachel05954619,,60,4099,None,False,False,2023,None,None,640,0
1199,2021-03-11 20:17:14,1370106564565155849,RT @NBCNews: BREAKING: President Biden signs $...,244405579,GloRIHusBadGaL,☈_Planet420_☈,564,39301,None,False,False,35899,None,None,2365,0


### Clean tweets

In [348]:
# not working when I put these into a function
tweets_df = tweets_df.drop_duplicates(subset=['id_str'])
tweets_df = tweets_df.dropna(subset=['id_str', 'screen_name'])

In [349]:
# Makes tweets more readable 
def CleanTweets(txt): 
    # remove @ username 
    # txt = re.sub(r'@[A-Za-z0-9)]+','', txt)
    # this removes names, which are sometimes vaccine names, so not good for my filter

    # remove @ from username
    txt = txt.replace('@','')    
    
    # remove RT    
    txt = re.sub(r'RT[\s]+','',txt)
    
    # remove # but leave txt
    txt = re.sub(r'#','', txt)
    
    # remove hyperlinks
    txt = re.sub(r'https?:\/\/\S+', '', txt)
    return txt

In [350]:
# Clean the text field 
tweets_df['text'] = tweets_df['text'].apply(CleanTweets)

In [351]:
# to get subjectivity 
def RateSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

In [352]:
# to get polarity
def RatePolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [353]:
# add worded sentiment based on polarity score
def GetSentiment(num):
    if num < 0: 
        return 'negative' 
    elif num == 0:
        return 'neutral'
    else:
        return 'positive'

In [354]:
# Create Columns
def RateTweets(df):
    df['Subjectivity'] = df['text'].apply(RateSubjectivity)
    df['Polarity'] = df['text'].apply(RatePolarity)
    df['Sentiment'] = df['Polarity'].apply(GetSentiment)
    return df 

In [355]:
tweets_df = RateTweets(tweets_df)
tweets_df.to_csv(output_csv, index=False, encoding='UTF-8')
tweets_df.tail(3)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count,Subjectivity,Polarity,Sentiment
1197,2021-03-11 20:17:15,1370106565253070850,MichelleObama: When you can get the COVID-19 v...,2336674833,Ritmoyclase2,,473,184548,None,True,False,190270,None,None,8413,0,0.65,0.75,positive
1198,2021-03-11 20:17:15,1370106565190094849,cantpretendtoo: Wife’s Uncle got the Covid vac...,1250179015215898624,Rachel05954619,,60,4099,None,False,False,2023,None,None,640,0,1.00,-0.50,negative
1199,2021-03-11 20:17:14,1370106564565155849,NBCNews: BREAKING: President Biden signs $1.9T...,244405579,GloRIHusBadGaL,☈_Planet420_☈,564,39301,None,False,False,35899,None,None,2365,0,0.00,0.00,neutral


In [321]:
# tweets_df[['created','id_str','text','screen_name','location','Subjectivity','Polarity','Sentiment']].tail(5)

# SQLAlchemy
### Connecting to a certain other host 

In [5]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [12]:
engine = create_engine(conn, echo=False)

In [13]:
Base = automap_base()
Base.prepare(engine, reflect=True)  

In [14]:
# Test connection
# it works!!!
# not anymore
# kill kernal if conn is wrong

Base.classes.keys()

['tw2']

### Don't run yet
Clean up your tweets! 

In [9]:
_df = pd.read_csv('csvs/tw2.csv')
_df.to_sql(name='tw2', con=engine, if_exists='append', index=True)
print('added!')

added!


In [ ]:
# session things
session = Session(engine)
session.close()


In [344]:
import schedule
import time

In [ ]:
def EveryFifteen():
    print('time')
    
schedule.every(15).minutes
